In [1]:
import pandas as pd
import itertools

## Create database retrieval

In [10]:
df = pd.read_csv('../data/filtered_df_v2.csv')
train = pd.read_csv('../data/train.csv')

In [14]:
df['fname'] = df['path'].apply(lambda x: x.split('/')[-1])

0        6815.jpg
1        7024.jpg
2        7053.jpg
3        7364.jpg
4        7361.jpg
          ...    
2467    40982.jpg
2468    40986.jpg
2469    28322.jpg
2470    34715.jpg
2471    34206.jpg
Name: fname, Length: 2472, dtype: object

In [15]:
merged = pd.merge(df, train[['name', 'group']], how='left', left_on='fname', right_on='name')

In [16]:
classes = merged['class'].unique()

In [17]:
classes

array(['trousers', 'shoe', 'shorts', 'jacket', 'sweatshirt',
       'elegant_jacket', 'high_heels_shoe', 't_shirt', 'bag'],
      dtype=object)

In [19]:
dataset = {}
for cls in classes:
    dataset[cls] = list(merged[merged['class'] == cls]['group'].value_counts().keys())

In [20]:
retrieval_base = pd.DataFrame(columns=['name', 'class', 'group'])
for key in dataset.keys():
    retrieval_base = retrieval_base.append(train[train['group'].isin(dataset[key])][['name', 'class', 'group']])

In [21]:
retrieval_base.drop_duplicates(['name', 'class', 'group'], inplace=True)

In [22]:
retrieval_base = retrieval_base.sort_index()

In [25]:
retrieval_base.to_csv('../data/retrieval_base.csv', sep=',', index=False)

### SUPERFILTERED

In [26]:
retrieval_base_filtered = pd.DataFrame(columns=['name', 'class', 'group'])

In [27]:
classes = set(retrieval_base['class'].values)

In [28]:
for cls in classes:
    subset = retrieval_base[retrieval_base['class'] == cls]
    if len(subset) > 3:
        retrieval_base_filtered = retrieval_base_filtered.append(subset.sample(3))

In [30]:
retrieval_base_filtered.to_csv('../data/retrieval_base_filtered.csv', sep=',', index=False)